In [ ]:
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from pymongo import MongoClient
from uri import uri
from password import password

In [ ]:
Login = "study.ai_172@mail.ru"
password =  password
split_login = Login.split('@')[0]

In [ ]:
chrome_options = Options()
chrome_options.add_argument("--start-fullscreen")

In [ ]:
driver = webdriver.Chrome(
    executable_path=os.popen('which chromedriver').read().strip(),
    options=chrome_options)

In [ ]:
driver.get('https://mail.ru')

In [ ]:
Fill_login = driver.find_element_by_xpath('//input[@name="login"]')

In [ ]:
Fill_login.send_keys(split_login)
Fill_login.send_keys(Keys.ENTER)

In [ ]:
time.sleep(2)

In [ ]:
Fill_password = driver.find_element_by_xpath('//input[@type="password"]')
Fill_password.send_keys(password)
Fill_password.send_keys(Keys.ENTER)

In [ ]:
wait_period = WebDriverWait(driver,15)
wait_period.until(EC.element_to_be_clickable((By.CLASS_NAME, 'js-tooltip-direction_letter-bottom')))

In [ ]:
Total_email = set()

In [ ]:
while True:
    current_length = len(Total_email)
    get_links = driver.find_elements_by_class_name('js-tooltip-direction_letter-bottom')
    
    for link in get_links:
        Total_email.add(link.get_attribute('href'))
        
    actions = ActionChains(driver)
    actions.move_to_element(get_links[-1])
    actions.perform()
    
    if len(Total_email) == current_length:
        break

In [ ]:
uri = uri
client = MongoClient(uri,27017)
db = client["task"]
mail = db['email']

In [ ]:
for link in Total_email:
    driver.get(link)
    
    wait = WebDriverWait(driver,10)
    sender = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'letter-contact')))
    
    info = {
        'from':sender.get_attribute('title'),
        'date':driver.find_element_by_xpath('//div[@class="letter__date"]').text,
        'title':driver.find_element_by_xpath('//h2[@class="thread__subject"]').text,
        'message':driver.find_element_by_xpath('//div[@class="letter-body__body"]').text,
        'link':link
    }
    
    mail.update_one(
        {'link':info['link']},
        {'$set':info},upsert = True
    )
    